
# Make PyTorch GT for reference
Only Python 3.7 + pytorch11.7+cu117 is tested. Due to limited support of Softsplat op, other Python version may not work. 

In [6]:
# env
import os
import cv2
import numpy as np
import time


In [15]:
# make gt inference (540p) check.png
# The inference.py is used to generate GMFSS inferenced output as gt for later comparison 
start = time.time()
assert os.system("python inference.py") == 0
end = time.time()
print("Time elapsed: ", end - start)

Time elapsed:  8.720263004302979


In [9]:
# compare ref
# read check.png and ref.png and compare them with np
out = cv2.imread("check.png")
ref = cv2.imread("ref.png")
np.allclose(out, ref, atol=1, equal_nan=True)

True

# Export Traced TorchScript and use PNNX to export ncnn.param, ncnn.bin
GMFSS(PG104, or Fortuna) is split into ReuseNet and InferNet. 
ReuseNet generates GMFlow and feature pyramids.
InferNet performs Softsplat operation using output from ReuseNet and generates final output. 


```

(reusenet)
.\pnnx.exe D:/60-fps-Project/VFI/GMFSS2NCNN/reuse_576.pt inputshape=[1,3,576,960]f32,[1,3,576,960]f32 device=cpu moduleop=model.gmflow.utils.split_feature,model.gmflow.utils.merge_splits,model.gmflow.utils.convex_upsampling                      

(infernet, note that pnnx's option customop contradicts with moduleop)
.\pnnx.exe D:/60-fps-Project/VFI/GMFSS2NCNN/infer_576.pt inputshape=[1,3,576,960]f32,[1,3,576,960]f32,[1,1,1,1]f32,[1,2,288,480]f32,[1,2,288,480]f32,[1,2,288,480]f32,[1,64,288,480]f32,[1,128,144,240]f32,[1,192,72,120]f32,[1,64,288,480]f32,[1,128,144,240]f32,[1,192,72,120]f32 device=gpu moduleop=model.gmflow.utils.split_feature,model.gmflow.utils.merge_splits,model.gmflow.utils.convex_upsampling,model customop=D:\60-fps-Project\VFI\GMFSS2NCNN\softsplat_cuda.pyd       

(todo)
ncnnoptimize.exe D:\60-fps-Project\VFI\GMFSS2NCNN\flownet_288.ncnn.param D:\60-fps-Project\VFI\GMFSS2NCNN\flownet_288.ncnn.bin D:\60-fps-Project\VFI\GMFSS2NCNN\flownet_288.ncnn.opt.param D:\60-fps-Project\VFI\GMFSS2NCNN\flownet_288.ncnn.opt.bin 1

(no use)
.\pnnx.exe D:/60-fps-Project/VFI/GMFSS2NCNN/flownet_288.pt inputshape=[1,3,288,480]f32,[1,3,288,480]f32 device=cpu moduleop=model.gmflow.utils.split_feature,model.gmflow.utils.merge_splits,model.gmflow.utils.convex_upsampling

```

copy generated bin to model dir of gmfss-demo

In [24]:
# (should be useless)
# compare pt with ref pt
import numpy as np
output = np.load("output.npy")
ref = np.load("ref_output.npy")
np.allclose(output, ref, atol=1, equal_nan=True)


False

## Clean pnnx.py

In [ ]:
# Useless
with open("flownet_288_pnnx.py", 'r', encoding='utf-8') as r:
    content = r.read()
content = content.replace("", "")
with open("flownet_288_pnnx.py", 'w', encoding='utf-8') as w:
    w.write(content)


# Edit NCNN Param

The following part replaces incorrect node from PNNX (TorchScript -> NCNN Param).
You don't need to understand the code(or you can), just execute and it'll generate usable ncnn.param.
```python
(Don't try to understand notes below):
Replace InstanceNorm
```
nn\.InstanceNorm2d(\s+)(.*?)(\s+)([\d+\s]+)
InstanceNorm$1$2$3$4 2=0
```

after:
```python
v_964 = F.upsample(input=v_963, align_corners=True, mode='bilinear', scale_factor=(2.000000,2.000000))
v_965 = (v_964 * 2)
```
all split, merge, s=8

In [11]:
# reusenet
import os, shutil, re
param_path = "reuse_576.ncnn.param"
param_bak_path = "reuse_576.ncnn_bak.param"

if os.path.exists(param_bak_path):
    os.remove(param_bak_path)
shutil.copy(param_path, param_bak_path)
content = ""
is_s_8 = False
with open(param_bak_path, 'r', encoding='utf-8') as r:
    for l in r.readlines():
        if re.search("nn.InstanceNorm2d", l, re.I):
                l = l.replace("nn.InstanceNorm2d", "InstanceNorm")
                l = f"{l.strip()} 2=0\n"
        if "upsample_122" in l:  # flow01 s=8 part starts
            is_s_8 = True
        if "model.gmflow.utils.convex_upsampling convex_upsampling" in l:  # flow01 over, flow10 s=2 part starts
            is_s_8 = False
        if "upsample_123" in l: # flow10 s=8 part starts
            is_s_8 = True
        if is_s_8:
            if re.search("split_feature|merge_splits", l, re.I):
                l = f"{l.strip()} 0=8\n"
        if re.search("convex_upsampling", l, re.I):
            l = f"{l.strip()} 0=4\n"
        content += l

with open(param_path, 'w', encoding='utf-8') as w:
    w.write(content)

In [87]:
# infernet
import os, shutil, re
param_path = "infer_576.ncnn.param"
param_bak_path = "infer_576.ncnn_bak.param"

if os.path.exists(param_bak_path):
    os.remove(param_bak_path)
shutil.copy(param_path, param_bak_path)
content = ""

# edit input tag
with open(param_bak_path, 'r', encoding='utf-8') as r:
    # img0, img1, timestep, flow01, flow10, metric, feat11, feat12, feat13, feat21, feat22, feat23
    content = r.read()
    content = content.replace(" in10", " feat22")
    content = content.replace(" in11", " feat23")
    content = content.replace(" in0", " img0")
    content = content.replace(" in1", " img1")
    content = content.replace(" in2", " timestep")
    content = content.replace(" in3", " flow01")
    content = content.replace(" in4", " flow10")
    content = content.replace(" in5", " metric")
    content = content.replace(" in6", " feat11")
    content = content.replace(" in7", " feat12")
    content = content.replace(" in8", " feat13")
    content = content.replace(" in9", " feat21")
    
# replace softsplat ir pattern
ss_pattern = r"(softsplat\.forward.*?2 1 \d+ \d+ )\d+\nSplit.*?\nCrop.*?\nReshape.*?\nBinaryOp.*?\nReshape.*?\nCrop.*?\nBinaryOp.*?(\d+) 0=3\n"
ss_replacement = r"\1\2\n"
content = re.sub(ss_pattern, ss_replacement, content, flags=re.DOTALL)

contents = content.splitlines()
contents[1] = f"{len(contents) - 2} {contents[1].split(' ')[1]}"
content = "\n".join(contents)

with open(param_path, 'w', encoding='utf-8') as w:
    w.write(content)

In [36]:
# search to locate debug output
with open("infer_576.ncnn.param", 'r', encoding='utf-8') as r:
    for l in r.readlines():
        if 'softsplat' in l:
            print(l.split('        ')[1].strip())

pnnx_125
pnnx_171
pnnx_219
pnnx_267
pnnx_315
pnnx_363
pnnx_411
pnnx_459


In [83]:
# below for debug or test, don't need to execute
# test ss pattern
input_ir = """
softsplat.forward        pnnx_125                 2 1 54 20 55
Split                    splitncnn_7              1 2 55 56 57
Crop                     select_0                 1 1 56 58 -23310=1,0 -23311=1,0 -23309=1,-1
Reshape                  reshape_204              1 1 58 59 0=480 1=288
BinaryOp                 add_12                   1 1 59 60 0=0 1=1 2=1.000000e-05
Reshape                  reshape_205              1 1 60 61 0=480 1=288
Crop                     slice_222                1 1 57 62 -23310=1,-1 -23311=1,0 -23309=1,0
BinaryOp                 div_13                   2 1 62 61 63 0=3
Crop                     slice_223                1 1 32 64 -23310=1,2 -23311=1,0 -23309=1,1
Split                    splitncnn_8              1 3 64 65 66 67
UnaryOp                  exp_14                   1 1 65 68 0=7
Split                    splitncnn_9              1 4 68 69 70 71 72
Interp                   upsample_14              1 1 img1 73 0=2 1=5.000000e-01 2=5.000000e-01 6=0
Split                    splitncnn_10             1 2 73 74 75
BinaryOp                 mul_15                   2 1 75 71 76 0=2
Concat                   cat_1                    2 1 76 69 77 0=0
softsplat.forward        pnnx_171                 2 1 77 26 78
Split                    splitncnn_11             1 2 78 79 80
Crop                     select_1                 1 1 79 81 -23310=1,0 -23311=1,0 -23309=1,-1
Reshape                  reshape_206              1 1 81 82 0=480 1=288
BinaryOp                 add_16                   1 1 82 83 0=0 1=1 2=1.000000e-05
Reshape                  reshape_207              1 1 83 84 0=480 1=288
Crop                     slice_224                1 1 80 85 -23310=1,-1 -23311=1,0 -23309=1,0
BinaryOp                 div_17                   2 1 85 84 86 0=3
"""
pattern = r"(softsplat\.forward.*?2 1 \d+ \d+ )\d+\nSplit.*?\nCrop.*?\nReshape.*?\nBinaryOp.*?\nReshape.*?\nCrop.*?\nBinaryOp.*?(\d+) 0=3\n"
replacement = r"\1\2\n"
print(re.sub(pattern, replacement, input_ir, flags=re.DOTALL))


softsplat.forward        pnnx_125                 2 1 54 20 63
Crop                     slice_223                1 1 32 64 -23310=1,2 -23311=1,0 -23309=1,1
Split                    splitncnn_8              1 3 64 65 66 67
UnaryOp                  exp_14                   1 1 65 68 0=7
Split                    splitncnn_9              1 4 68 69 70 71 72
Interp                   upsample_14              1 1 img1 73 0=2 1=5.000000e-01 2=5.000000e-01 6=0
Split                    splitncnn_10             1 2 73 74 75
BinaryOp                 mul_15                   2 1 75 71 76 0=2
Concat                   cat_1                    2 1 76 69 77 0=0
softsplat.forward        pnnx_171                 2 1 77 26 86



# (Debug) Compare layer output with torchscript output (gt)

In [58]:
# reusenet
import re
import numpy as np
torch_output_mat = r"D:\60-fps-Project\VFI\GMFSS2NCNN\debug6_480.288.1.65_ss_in.txt"
ncnn_output_mat = r"D:\Program\VSsource\research\gmfss-ncnn-vulkan-demo\src\out\build\x64-Debug\debug10_480.288.1.65_pnnx_267_buttom0.txt"
with open(ncnn_output_mat, 'r', encoding='utf-8') as r:
    print(r.read()[:1000])
with open(torch_output_mat, 'r', encoding='utf-8') as r:
    print(r.read()[:1000])

shape 480 288 1 65
0.314 0.179 0.215 0.203 0.213 0.203 0.205 0.204 0.199 0.191 0.184 0.183 0.184 0.187 0.192 0.195 0.205 0.211 0.213 0.238 0.280 0.338 0.290 0.254 0.211 0.237 0.327 0.398 0.410 0.439 0.401 0.371 0.360 0.398 0.418 0.423 0.419 0.423 0.418 0.401 0.406 0.396 0.398 0.389 0.363 0.390 0.391 0.387 0.405 0.433 0.459 0.473 0.477 0.472 0.465 0.437 0.424 0.441 0.440 0.424 0.413 0.408 0.405 0.410 0.407 0.405 0.396 0.407 0.406 0.400 0.394 0.397 0.409 0.425 0.460 0.442 0.415 0.395 0.408 0.384 0.380 0.381 0.381 0.378 0.376 0.367 0.365 0.367 0.358 0.363 0.368 0.377 0.386 0.342 0.403 0.403 0.373 0.344 0.331 0.330 0.329 0.331 0.333 0.332 0.336 0.340 0.339 0.332 0.325 0.327 0.332 0.337 0.363 0.382 0.372 0.388 0.325 0.312 0.341 0.359 0.358 0.359 0.357 0.358 0.371 0.395 0.402 0.396 0.396 0.422 0.470 0.556 0.496 0.437 0.406 0.356 0.257 0.206 0.131 0.105 0.102 0.108 0.126 0.154 0.178 0.201 0.216 0.214 0.192 0.194 0.268 0.416 0.494 0.330 0.269 0.302 0.338 0.358 0.353 0.360 0.383 0.378 0.445 0.4

In [74]:
import re
import numpy as np
torch_output_mat = r"D:\60-fps-Project\VFI\GMFSS2NCNN\debug0_480.288.1.3_I2t.txt"
ncnn_output_mat = r"D:\Program\VSsource\research\gmfss-ncnn-vulkan-demo\src\out\build\x64-Debug\debug6_480.288.1.3_div_13_top0.txt"

class PrintData:
    def __init__(self, r) -> None:
        shape_info = r.readline()
        shape_info = re.findall(r"shape (\d+) (\d+) (\d+) (\d+)", shape_info)[0]
        self.w, self.h, self.d, self.c = map(int, shape_info)
        data = r.readline().split(' ')
        data_ = list()
        for d in data:
            try:
                data_.append(float(d))
            except:
                print(f"failing: {d}")
        self.data = np.array(data_).astype(np.float32)
        self.pad = self.w * self.h * self.d * self.c - len(data_)
        self.data = np.pad(self.data, ((0, self.pad),), 'constant', constant_values=(0, 0))
        self.data = self.data[:self.w * self.h * self.d * self.c].reshape(self.c, self.d, self.h, self.w)
        print(f"pad {self.pad}")

with open(torch_output_mat, 'r', encoding='utf-8') as r:
    torch_output = PrintData(r)
with open(ncnn_output_mat, 'r', encoding='utf-8') as r:
    ncnn_output = PrintData(r)

cp_cnt = 10000
np.abs(torch_output.data.flatten()[:cp_cnt] - ncnn_output.data.flatten()[:cp_cnt]).mean()

failing: 
pad 0
failing: 
pad 0


5625211.5

In [59]:
print(torch_output.data[0, 0, 0, 30])
print(ncnn_output.data[0, 0, 0, 30])
# torch_output.data.shape

0.545
0.392


# (Debug) Param Conversion Playground

In [ ]:
import torch
torch.load("train_log/flownet.pkl")

In [4]:
import torch.nn.functional as F
t1 = torch.tensor([[1, 2, 3], [4, 5, 6]]).float()
t2 = torch.tensor([[3, 2, 1], [6, 5, 4]]).float()
t3 = F.l1_loss(t1, t2, reduction='none')
print(t3)
t3.mean(1, True)

tensor([[2., 0., 2.],
        [2., 0., 2.]])


tensor([[1.3333],
        [1.3333]])

In [7]:
import pickle
with open("reuse_output.pkl", 'rb') as r:
    reuse_output = pickle.load(r)
print(','.join([f"[{i.shape[0]},{i.shape[1]},{i.shape[2]},{i.shape[3]}]f32" for i in reuse_output]))
print(','.join([f"{i.shape}" for i in reuse_output]))


[1,2,288,480]f32,[1,2,288,480]f32,[1,1,288,480]f32,[1,1,288,480]f32,[1,64,288,480]f32,[1,128,144,240]f32,[1,192,72,120]f32,[1,64,288,480]f32,[1,128,144,240]f32,[1,192,72,120]f32
torch.Size([1, 2, 288, 480]),torch.Size([1, 2, 288, 480]),torch.Size([1, 1, 288, 480]),torch.Size([1, 1, 288, 480]),torch.Size([1, 64, 288, 480]),torch.Size([1, 128, 144, 240]),torch.Size([1, 192, 72, 120]),torch.Size([1, 64, 288, 480]),torch.Size([1, 128, 144, 240]),torch.Size([1, 192, 72, 120])


# (Debug) Op Playground

In [2]:
import torch
torch.ops.load_library("softsplat_cuda.pyd")
torch.ops.softsplat.forward

<OpOverloadPacket(op='softsplat.forward')>

In [67]:
import torch
from model.gmflow.utils import split_feature, merge_splits, convex_upsampling
# generate (2,2,1,2) tensor
feat = torch.arange(0, 72).reshape(1, 2, 6, 6)
splits = split_feature()(feat, 2)
print(splits)

merged = merge_splits()(splits, 2)
print(merged)


tensor([[[[[ 0,  1,  2],
           [ 6,  7,  8],
           [12, 13, 14]],

          [[36, 37, 38],
           [42, 43, 44],
           [48, 49, 50]]],


         [[[ 3,  4,  5],
           [ 9, 10, 11],
           [15, 16, 17]],

          [[39, 40, 41],
           [45, 46, 47],
           [51, 52, 53]]],


         [[[18, 19, 20],
           [24, 25, 26],
           [30, 31, 32]],

          [[54, 55, 56],
           [60, 61, 62],
           [66, 67, 68]]],


         [[[21, 22, 23],
           [27, 28, 29],
           [33, 34, 35]],

          [[57, 58, 59],
           [63, 64, 65],
           [69, 70, 71]]]]])
tensor([[[[ 0,  1,  2,  3,  4,  5],
          [ 6,  7,  8,  9, 10, 11],
          [12, 13, 14, 15, 16, 17],
          [18, 19, 20, 21, 22, 23],
          [24, 25, 26, 27, 28, 29],
          [30, 31, 32, 33, 34, 35]],

         [[36, 37, 38, 39, 40, 41],
          [42, 43, 44, 45, 46, 47],
          [48, 49, 50, 51, 52, 53],
          [54, 55, 56, 57, 58, 59],
          [60,

In [68]:
flow = torch.arange(0, 12).reshape(1, 2, 2, 3)
up_flow = torch.arange(0, 48).reshape(1, 2, 2*2, 2*3)
print(up_flow)
up_flow = convex_upsampling(2)(flow, up_flow)
print(up_flow)



tensor([[[[ 0,  1,  2,  3,  4,  5],
          [ 6,  7,  8,  9, 10, 11],
          [12, 13, 14, 15, 16, 17],
          [18, 19, 20, 21, 22, 23]],

         [[24, 25, 26, 27, 28, 29],
          [30, 31, 32, 33, 34, 35],
          [36, 37, 38, 39, 40, 41],
          [42, 43, 44, 45, 46, 47]]]])
tensor([[[[ 0,  6,  1,  7,  2,  8],
          [12, 18, 13, 19, 14, 20],
          [ 3,  9,  4, 10,  5, 11],
          [15, 21, 16, 22, 17, 23]],

         [[24, 30, 25, 31, 26, 32],
          [36, 42, 37, 43, 38, 44],
          [27, 33, 28, 34, 29, 35],
          [39, 45, 40, 46, 41, 47]]]])


In [6]:
import torch
import torchvision.models as models

net = models.resnet18(pretrained=False)
net = net.eval()

x = torch.rand(1, 3, 224, 224)

# You could try disabling checking when tracing raises error
# mod = torch.jit.trace(net, x, check_trace=False)
mod = torch.jit.trace(net, x)

mod.save("resnet18.pt")

c:\Users\Administrator\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Administrator\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
